# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [1]:
# Instalace spaCy z Jupyter Notebooku
import sys

import numpy as np
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

     |████████████████████████████████| 12.0 MB 6.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Lemmatizace textu př.:
# " ".join([token.lemma_ for token in lemmatizer(text)])

In [3]:
import pandas as pd

df = pd.read_csv('articles.csv')
df.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


In [4]:
import string

def lower(s: str) -> str:
    return s.lower()

def strip_punctuation(s: str) -> str:
    return s.translate(str.maketrans('', '', string.punctuation))

def lemmanize(s: str) -> str:
    return ' '.join([token.lemma_ for token in lemmatizer(s)])

def preprocess(df: pd.DataFrame) -> None:
    for column in ['author', 'title', 'text', 'claps']:
        df[column] = df[column].apply(lower).apply(strip_punctuation).apply(lemmanize)

In [5]:
preprocess(df)
df.head()

,author,claps,reading_time,link,title,text
0,justin lee,83k,11,https://medium.com/swlh/chatbots-were-the-next...,chatbot be the next big thing what happen – th...,oh how the headline blare \n chatbot be the ne...
1,conor dewey,14k,7,https://towardsdatascience.com/python-for-data...,python for data science 8 concept -PRON- may h...,if -PRON- have ever find -PRON- look up the sa...
2,william koehrsen,28k,11,https://towardsdatascience.com/automated-featu...,automate feature engineering in python – towar...,machine learning be increasingly move from han...
3,gant laborde,13k,7,https://medium.freecodecamp.org/machine-learni...,machine learn how to go from zero to hero – fr...,if -PRON- understanding of ai and machine lear...
4,emmanuel ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,reinforcement learn from scratch – insight datum,want to learn about applied artificial intelli...


## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [6]:
def make_index(data):
    word_to_idx = {}
    for idx, title in enumerate(data):
        for word in title.split():
            if not word in word_to_idx:
                word_to_idx[word] = {idx}
            else:
                word_to_idx[word].add(idx)

    return word_to_idx

In [7]:
title_word_to_idx = make_index(df.title)
text_word_to_idx = make_index(df.text)

In [8]:
dict(list(title_word_to_idx.items())[:2])

{'chatbot': {0, 86, 96, 192, 223, 257, 267, 273},
 'be': {0,
  12,
  13,
  14,
  15,
  18,
  36,
  56,
  57,
  58,
  60,
  61,
  83,
  86,
  87,
  98,
  108,
  113,
  121,
  123,
  128,
  136,
  138,
  139,
  140,
  153,
  169,
  170,
  216,
  218,
  224,
  227,
  231,
  233,
  237,
  243,
  246,
  258,
  261,
  264,
  267,
  273,
  321,
  330,
  335}}

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [9]:
def tf_idf(query, document, num_docs, word_to_idx):
    normalized_query = lemmanize(strip_punctuation(lower(query))).strip()
    words_in_query = normalized_query.split()

    score = 0
    for word_in_query in set(words_in_query):
        if not word_in_query in word_to_idx:
            continue

        num_document_occurrences = document.count(word_in_query) # c(w, d)
        if num_document_occurrences == 0:
            continue

        num_query_occurrences = words_in_query.count(word_in_query) # c(w, q)
        num_word_in_documents = len(word_to_idx[word_in_query]) # df

        score += num_query_occurrences * num_document_occurrences * np.log((num_docs + 1) / num_word_in_documents)


    return score

## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [10]:
def make_score(titles, texts, title_word_to_idx, text_word_to_idx, alpha, query):
    assert len(titles) == len(texts)

    num_documents = len(titles)
    scores = np.zeros((num_documents, 1))

    for i in range(num_documents):
        title, text = titles[i], texts[i]
        scores[i] = alpha * tf_idf(query, title, num_documents, title_word_to_idx) + \
                (1 - alpha) * tf_idf(query, text, num_documents, text_word_to_idx)

    return scores

In [11]:
df['score'] = make_score(query='coursera vs udacity machine learning',
           titles=df.title,
           texts=df.text,
           title_word_to_idx=title_word_to_idx,
           text_word_to_idx=text_word_to_idx,
           alpha=0.7)

In [12]:
df.sort_values(by='score', ascending=False).head(10)[['title', 'text', 'score']]

,title,text,score
276,coursera vs udacity for machine learning – hac...,2018 be an exciting time for student of machin...,42.880391
143,every single machine learn course on the inter...,a year and a half ago i drop out of one of the...,34.781632
67,every single machine learn course on the inter...,a year and a half ago i drop out of one of the...,34.781632
99,every single machine learn course on the inter...,a year and a half ago i drop out of one of the...,34.781632
19,every single machine learn course on the inter...,a year and a half ago i drop out of one of the...,34.781632
47,machine learn in a week – learn new stuff – me...,get into machine learning ml can seem like an ...,11.244242
327,in defense of skepticism about deep learning –...,in a recent appraisal of deep learning marcus ...,9.346694
209,in defense of skepticism about deep learning –...,in a recent appraisal of deep learning marcus ...,9.346694
64,machine learn in a year – learn new stuff – me...,this be a follow up to an article i write last...,9.321382
36,machine learning be fun – adam geitgey – medium,update this article be part of a series check ...,8.150912
